In [11]:
from youtube_transcript_api import YouTubeTranscriptApi
from time import strftime
from time import gmtime
from tqdm import tqdm

In [12]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from bs4 import BeautifulSoup as bs

In [13]:
path = r"C:\Users\moham\Workplace\Freelancing\edgedriver_win64\msedgedriver.exe"

In [14]:
ser = Service(path)
options = webdriver.EdgeOptions()
options.add_argument('headless')

driver = webdriver.Edge(service = ser, options=options)

In [15]:
url = input('Enter Playlist URL: ')

Enter Playlist URL: https://www.youtube.com/playlist?list=PLBlnK6fEyqRgp46KUv4ZY69yXmpwKOIev


In [16]:
driver.get(url)
driver.implicitly_wait(5)
content = driver.page_source


In [17]:
driver.quit()

In [18]:
soup = bs(content, 'lxml')

In [19]:
def get_playlist_subtitles():
    videos = soup.find_all('a', {'id': 'video-title'})
    video_id = []
    video_title = []
    for idx, video in enumerate(videos):
        video_id.append(video['href'].split('&')[0].replace('/watch?v=', ''))
        video_title.append(str(idx+1) + ' - ' + video['title'].replace('|', '_'))
        
        
    for title, id_ in tqdm(zip(video_title, video_id)):
        try:
            srt = YouTubeTranscriptApi.get_transcript(video_id=id_)
            text = ''
            with open('subtitles/{}.srt'.format(title), 'w') as file:
                for idx, line in enumerate(srt):
                    text += str(idx+1) + '\n'
                    start_time = strftime("%H:%M:%S", gmtime(line['start']))
                    end_time = strftime("%H:%M:%S", gmtime(line['start'] + line['duration']))
                    text += start_time +' --> '+ end_time
                    text += '\n'
                    text += line['text']
                    text += '\n\n'
                file.write(text)
        except:
            continue

In [20]:
get_playlist_subtitles()

100it [01:37,  1.02it/s]
